# Python生成进度条

In [2]:
import numpy as np
import time
import torch
import tqdm

## 自定义格式化输出函数

In [3]:
# 进度条
def jindutiao():
    for i in range(1, 51):
        time.sleep(0.2)
        print('\r[%-51s]' % ('-' * i + '>'), end='')
    print()
for i in range(2):
    jindutiao()

[-------------------------------------------------->]
[-------------------------------------------------->]


## 使用tqdm工具包

In [41]:
# 使用进度条工具
def jindutiao():
    par = tqdm.tqdm(range(1, 51))
    for i in par:
        time.sleep(0.2)
for i in range(2):
    jindutiao()

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.94it/s]


## 使用progressbar工具包

In [39]:
import time
from progressbar import *
 
progress = ProgressBar()
for i in progress(range(1000)):
    time.sleep(0.01)

100% |########################################################################|


# softmax、logsoftmax、nllloss和CrossEntropyLoss之间的区别与联系

## Softmax

$$\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$$

## LogSoftmax
其实就是将Softmax取对数

$$\text{LogSoftmax}(x_{i}) = \log(\frac{\exp(x_i)}{\sum_j \exp(x_j)})$$

## NLLLoss
该函数全称是negative log likelihood

$$NLLLoss(x, class)=-x[class]$$

## CrossEntropyLoss

$$CrossEntropyLoss=\sum_k(p_k*\log q_k)$$
其中，p表示真实值，在这个公式中是ont-hot形式；q是预测值，在这里假设已经是经过softmax后的结果。

仔细观察可以知道，因为p的元素不是0就是1，而且又是乘法，所以很自然地我们如果知道1所对应的index，那么就不用做其他无意义的运算了。所以在pytorch代码中target不是以**one-hot**形式表示的，而是直接用**scalar**表示。所以交叉熵的公式(m表示真实类别)可变形为：
$$CrossEntropyLoss=\sum_k(p_k*\log q_k)=-log q_m $$
仔细观察不难发现，**CrossEntropyLoss**其实就是等同于**LogSoftmax**和**NLLLoss**两个步骤。

所以Pytorch中的**F.cross_entropy**会自动调用上面介绍的**LogSoftmax**和**NLLLoss**来计算交叉熵,其计算方式如下:
$$CrossEntropyLoss(x, class)=-log(\frac{\exp(x[class])}{\sum_j \exp(x[j])})$$

## 小结
在分类问题中，如果模型经过全连接后直接输出结果，那么在选用loss函数时应该选用CrossEntropyLoss；如果模型经过全连接后使用log_softmax函数，那么选用loss函数是应该选用NLLLoss。

## 代码实现演示

In [4]:
import torch
import torch.nn.functional as F
input_tensor = torch.randn(4,3)
input_tensor

tensor([[-1.0113, -0.2398, -1.1892],
        [-0.8600, -1.0444, -0.0218],
        [-1.1908,  1.5461,  0.1521],
        [-0.1471, -0.5752,  0.0168]])

### softmax

In [10]:
sfm = F.softmax(input_tensor)
sfmarray

<ipython-input-10-3ab1bf8f09c2>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sfm = F.softmax(input_tensor)


tensor([[0.2500, 0.5408, 0.2093],
        [0.2413, 0.2007, 0.5580],
        [0.0493, 0.7617, 0.1890],
        [0.3534, 0.2303, 0.4163]])

从结果可以看出，softmax函数得到的结果为一些概率值，并且每行tensor的和为1，故其结果十分有意义，可理解为某条记录为对应类别的概率。

举个例子：如果是图片分类任务，输入m张图片，输出为一个$m*N$ 的 tensor。上述中的tensor可以表示为输入4张图片，类别数为3.

再特殊一点，输入的四张图片，有3个类别分别为猫、狗和猪。可以看出第一张和第三张更有可能是狗。第二张和第四张更有可能是猪。

### logsoftmax

In [31]:
torch.log(sfm)

tensor([[-1.3863, -0.6148, -1.5642],
        [-1.4216, -1.6060, -0.5834],
        [-3.0091, -0.2722, -1.6662],
        [-1.0402, -1.4683, -0.8764]])

### nll_loss
假设标签值为**[1,2,1,2]**

In [29]:
# 利用公式手动计算
target=torch.tensor([1,2,1,2])
sum_nll = 0
for i in range(len(target)):
    sum_nll += -log_sfm[i][target[i]]
print(sum_nll/4)
(0.6148+0.5834+0.2722+0.8764)/4

tensor(0.5867)


0.5867

In [17]:
# 调用函数计算
target=torch.tensor([1,2,1,2])
F.nll_loss(log_sfm, target)

tensor(0.5867)

### cross_entropy_loss

In [30]:
F.cross_entropy(input_tensor, target)

tensor(0.5867)

# 拓展

## Softmax上溢和下溢问题

- 什么是上溢和下溢？
实数在计算机内用二进制表示，所以不是一个精确值，当数值过小的时候，被四舍五入为0，这个就是下溢问题。反之，当数值过大时就会出现上溢问题。
-softmax函数中需要进行指数运算$e^c$,当c极其大，$e^c$进行指数运算后会更大，导致计算机无法存储从而造成上溢。
-反义，当c及其小时，$e^c$就会无线趋近于0，当小数位到达一定的程度后，计算机就会将其四舍五入为0，导致下溢出。

In [40]:
inp = [1000.,500.,500.]
inp1 = [-1000., -1000., -1000.]

In [46]:
import math
import numpy as np
def softmax(inp):
    length = len(inp)
    exps = []
    res = 0
    ind = 0
    for item in inp:
        exp = math.exp(item)
        res = res + exp
        exps.append(exp)
        ind+=1
    exps = np.array(exps)   
    return exps/res

当指数函数里面传入的值很大时，就会出现上溢，爆出OverflowError.

In [47]:
softmax(inp)  # 上溢出

OverflowError: math range error

当指数函数里面传入的值是很小的负数时，就会出现下溢，输出结果就是0， 这样就有可能导致分母的值为0。

In [48]:
softmax(inp1)   # 下溢出

<ipython-input-46-cae78aadd3c7>:14: RuntimeWarning: invalid value encountered in true_divide
  return exps/res


array([nan, nan, nan])

## 解决办法
使用x-max(x)来代替x解决，这样就可以保证分子中最大值为0，并且分布中至少有一个1。

In [49]:
def softmax(inp):
    length = len(inp)
    exps = []
    res = 0
    ind = 0
    for item in inp:
        exp = math.exp(item-max(inp))
        res = res + exp
        exps.append(exp)
        ind+=1
    exps = np.array(exps)   
    return exps/res

In [50]:
softmax(inp)

array([1.00000000e+000, 7.12457641e-218, 7.12457641e-218])

In [51]:
softmax(inp1)

array([0.33333333, 0.33333333, 0.33333333])

可见上溢出和下溢出现象都已经解决。

## 使用pytorch查看是否会出现上溢和下溢问题

In [52]:
inp_tensor = torch.tensor(inp)
inp1_tensor = torch.tensor(inp1)
F.softmax(inp_tensor)

<ipython-input-52-4fe6a6246006>:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(inp_tensor)


tensor([1., 0., 0.])

In [53]:
F.softmax(inp1_tensor)

<ipython-input-53-2dd6e5c720df>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(inp1_tensor)


tensor([0.3333, 0.3333, 0.3333])

可见pytorch已经是经过处理了，不会出现上溢和下溢问题

此外，经过该处理后，计算logsoftmax值时，也不会出现log(0)$->$负无穷大的情况
$$\text{LogSoftmax}(x_{i}) = \log(\frac{\exp({x_i-M})}{\sum_j \exp(x_j-M)})$$
$$=log(\exp(x_j-M))-log(\sum_j\exp(x_j-M))=(x_j-M)-\log(\sum_j\exp(x_j-M))$$
此时，指数中的求和至少有一个为1.